# MLiP - K-Means Clustering Funktionsweise
Kurs Maschinelles Lernen in der Produktion

### In diesem Notebook wird das Schrittweise Vorgehen des K-Means Algorithmus gezeigt

Der K-Means Algorithmus versucht die within-cluster variation, also die Summe der quadrierten Abweichungen von den Cluster-Schwerpunkten (Centroids) zu minimieren.\
Da es zu Beginn noch keine Cluster gibt, müssen zuerst die Cluster, also eigentlich die Centroids initialisiert werden.\
Nun werden abwechselnd alle Datenpunkte den Cluster zugeordnet und anschließend die Centroide aktualisiert. 

### 0. Bibliotheken importieren

In [ ]:
# Importiere benötigte Bibliotheken
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
from matplotlib import colors

#Einstellungen für die Grafikausgabe
style = 'seaborn-whitegrid'
plt.style.use(style)
plt.rcParams.update({'font.size': 14})  # Schriftgröße aller Textzeichen im Graphen

# Definition eigener Farben für die spätere Darstellung
mycolors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
cmap = colors.ListedColormap(mycolors)

TODO:
* Wähle eine Zahl zwischen 1 und 100 für die Generierung deiner spezifischen Zufallszahlen my_seed=

(Wähle für alle Notebooks in allen Übungen immer die gleiche Zahl (z.B. den Tag deines Geburtstags), dann sind die Ergebnisse der verschiedenen Machine-Learning-Verfahren vergleichbar da dann alle Notebooks mit der "gleichen" Folge an Zufallszahlen arbeiten)

AUSGABE:
* Gewählte Zufallszahl

In [ ]:
# Erstelle eigene Zufallszahlen
my_seed = 2

# Ausgabe gewählte Zufallszahlen
print("\nGewählte Zahl für Zufallszahlen: \t" + str(my_seed))

# Seed für Zufallszahlengenerator von numpy festlegen
np.random.seed(my_seed)

### 1. Daten erfassen - Daten erzeugen

In [ ]:
# Datensatz erzeugen
df = pd.DataFrame({
    'x1': [8, 11, 15, 18, 22, 25, 27, 32, 30, 23, 27, 30, 42, 47, 49, 53, 42, 44, 46, 51, 59, 64, 71 ],
    'x2': [58, 57, 52, 67, 61, 50, 71, 65, 76, 19, 11, 6, 28, 10, 26, 3, 46, 42, 56, 45, 45, 61, 38]})

### 2. Daten erkunden

In [ ]:
plt.figure(figsize=(5, 5))
plt.scatter(df['x1'], df['x2'], color='k')
plt.xlim(0, 80)
plt.ylim(0, 80)
plt.xlabel('x1')
plt.ylabel('x2')
plt.show()

### 4. Modell bilden
In diesem Abschnitt werden nun zufällige Startwerte für die Centroiden erzeugt.\
Dann wird iterativ
* die Datenpunkten den Cluster zugeordnet
* die Centroiden neu für die Cluster berechnet

Dies geschieht so lange, bis sich keine Änderung bei den Clustern ergibt. 

In [ ]:
# Initialisierung durch 
# Festlegung der Anzahl der Cluster k
k = 3

# Zufällige Erzeugung der Centroiden (Mittelpunkte der Clusters)
centroids = pd.DataFrame({
    'x1': np.random.randint(0, 80, size=k),
    'x2': np.random.randint(0, 80, size=k),
    'cluster': np.arange(1,k+1)
    })

plt.figure(figsize=(8, 8))
plt.scatter(df['x1'], df['x2'], color='k')
for i in range(0,k):
    plt.plot(centroids.at[i,'x1'],centroids.at[i,'x2'], 's', markersize=8)
plt.xlim(0, 80)
plt.ylim(0, 80)
plt.xlabel('x1')
plt.ylabel('x2')
plt.legend([mpatches.Patch(color=cmap(b)) for b in np.arange(0,k)],
           ['Cluster '+ str(b) for b in np.arange(1,k+1)])
plt.title('Initialisierung Centroide')
plt.show()

### Ordne Daten in Cluster, d.h. den nächsten Centroiden zu

In [ ]:
# Berechne Abstand zu den Centroiden und bestimme nähesten Centroid
# Abstandsberechnungen
for i in range(0,k):
    # || x - c_i||^2 
    df['Abstand_c_'+str(i)] = np.sqrt((df['x1'] - centroids.at[i,'x1']) ** 2 
                                      + (df['x2'] - centroids.at[i,'x2']) ** 2)
# Bestimmung nähester Centroid
Abstand_cols = ['Abstand_c_'+str(i) for i in range(0,k)]
df['centroid'] = df.loc[:, Abstand_cols].idxmin(axis=1).apply(lambda x: int(x.split(sep='_')[-1]))
df['color'] = [mycolors[x] for x in df['centroid']]

# Plot Datenpunkte mit Einfärbung gemäß nähesten Centroid
fig = plt.figure(figsize=(8, 8))
plt.scatter(df['x1'], df['x2'], color=df['color'])
for i in range(0,k):
    plt.plot(centroids.at[i,'x1'],centroids.at[i,'x2'], 's', markersize=8)
plt.xlim(0, 80)
plt.ylim(0, 80)
plt.xlabel('x1')
plt.ylabel('x2')
plt.legend([mpatches.Patch(color=cmap(b)) for b in np.arange(0,k)],
           ['Cluster '+ str(b) for b in np.arange(1,k+1)])
plt.title('Erste Clusterzuordnung')
plt.show()

## Folgende beiden Schritte müssen iterativ nacheinander ausgeführt werden!

### Aktualisiere und Plotte neue Centroids

In [ ]:
# Plot Verschiebung der Centroids

# Speicherung alter Centroiden
centroids_alt = centroids.copy(deep=True)

# Aktualisierung der Centroids
for i in range(0,k):
    centroids.at[i,'x1'] = np.mean(df[df['centroid'] == i]['x1'])
    centroids.at[i,'x2'] = np.mean(df[df['centroid'] == i]['x2'])

# Plot Verschiebung Centroids
plt.figure(figsize=(8, 8))
plt.scatter(df['x1'], df['x2'], color=df['color'], edgecolor='k')
for i in range(0,k):
    plt.plot(centroids.at[i,'x1'], centroids.at[i,'x2'], 's', markersize=8)
plt.xlim(0, 80)
plt.ylim(0, 80)
plt.xlabel('x1')
plt.ylabel('x2')
for i in range(0,k):
    x1_alt = centroids_alt.at[i,'x1']
    x2_alt = centroids_alt.at[i,'x2']
    dx1 = (centroids.at[i,'x1'] - centroids_alt.at[i,'x1']) * 0.75
    dx2 = (centroids.at[i,'x2'] - centroids_alt.at[i,'x2']) * 0.75
    plt.arrow(x1_alt, x2_alt, dx1, dx2, head_width=2, head_length=3, fc=mycolors[i], ec=mycolors[i])
plt.legend([mpatches.Patch(color=cmap(b)) for b in np.arange(0,k)],
           ['Cluster '+ str(b) for b in np.arange(1,k+1)])
plt.title('Update Centroide')
plt.show()

### Ordne Daten in Cluster, d.h. den nächsten Centroiden zu

In [ ]:
# Abstandsberechnungen
for i in range(0,k):
    # || x - c_i||^2 
    df['Abstand_c_'+str(i)] = np.sqrt((df['x1'] - centroids.at[i,'x1']) ** 2 
                                      + (df['x2'] - centroids.at[i,'x2']) ** 2)
# Bestimmung nähester Centroid
Abstand_cols = ['Abstand_c_'+str(i) for i in range(0,k)]
df['centroid_alt'] = df['centroid']
df['color_alt'] = df['color']
df['centroid'] = df.loc[:, Abstand_cols].idxmin(axis=1).apply(lambda x: int(x.split(sep='_')[-1]))
df['color'] = [mycolors[x] for x in df['centroid']]

# Plot Datenpunkte mit Einfärbung (alt vs. neu) gemäß nähesten Centroid 
plt.figure(figsize=(16, 8))

# Plot alte Zuordnung
plt.subplot(1, 2, 1)
plt.scatter(df['x1'], df['x2'], color=df['color_alt'])
for i in range(0,k):
    plt.plot(centroids.at[i,'x1'],centroids.at[i,'x2'], 's',markersize=8)
plt.xlim(0, 80)
plt.ylim(0, 80)
plt.xlabel('x1')
plt.ylabel('x2')
plt.legend([mpatches.Patch(color=cmap(b)) for b in np.arange(0,k)],
           ['Cluster '+ str(b) for b in np.arange(1,k+1)])
plt.title('Clusterzuordnung alt')

# Plot neue Zuordnung
plt.subplot(1, 2, 2)

plt.scatter(df['x1'], df['x2'], color=df['color'])
for i in range(0,k):
    plt.plot(centroids.at[i,'x1'], centroids.at[i,'x2'], 's', markersize=8)
plt.xlim(0, 80)
plt.ylim(0, 80)
plt.xlabel('x1')
plt.ylabel('x2')
plt.legend([mpatches.Patch(color=cmap(b)) for b in np.arange(0,k)],
           ['Cluster '+ str(b) for b in np.arange(1,k+1)])
plt.title('Clusterzuordnung neu')
plt.show()